# C/C++ Library with Python 

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. 

But for certain types of calculations Python (and any other interpreted language) can be **very slow**.

Such calculations may be implemented in a compiled language such as C or Fortran.

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

## 1 ctypes - access the C library

**ctypes** is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

http://docs.python.org/3/library/ctypes.html

We need to load the library and set properties such as the functions return and argument types using the **ctypes** package :

1. **Loads the library** 

  **ctypes** exports the **cdll**, and on Windows **windll** objects, for loading dynamic link libraries.

   * **cdll.LoadLibrary(name)** : loads the library  which export functions using standard `__cdecl` calling convention
   * **windll.LoadLibrary(name)** : loads the library with `__stdcall` calling convention for the function 


2. **Specifying the required `argument` types:`argtypes`**  
  
  * It specify the required argument types of functions exported from DLLs by setting the argtypes attribute

3. **Return types: `restype`**

  * Return typescan be specified by setting the restype attribute of the function object.

4. **Function prototypes**
 
 * **CFUNCTYPE** 
 
   The CFUNCTYPE factory function creates types for callback functions using the normal cdecl calling convention
   
```python
ctypes.CFUNCTYPE(restype, *argtypes, use_errno=False, use_last_error=False)
``` 
 * **WINFUNCTYPE**
  
  Windows only: The returned function prototype creates functions that use the `__stdcall` calling convention
```python
ctypes.WINFUNCTYPE(restype, *argtypes, use_errno=False, use_last_error=False)
```

### Call the Shared Library in from Python

#### 1 libdemodll.dll 
 
###### 1.1 C code 
calling convention

* `__cdecl` :

```c
int GetNumber(int *v)
```
 
* `__stdcall: 

```c
__declspec(dllexport) int __stdcall  GetNumber(int *v)
```

In [4]:
%%file ./demo/src/demodll.c

int GetNumber(int *v); 

//__declspec(dllexport) int __stdcall  GetNumber(int *v); 

int GetNumber(int *v)
{
    int temp;
	temp=*v+1;
	*v=temp;
	return (*v)+1;
}

Overwriting ./demo/src/demodll.c


##### Build DLL 

In [1]:
!gcc -c -O3 -Wall -fPIC -o ./demo/bin/demodll.o  ./demo/src//demodll.c
!gcc -shared -o ./demo/bin/libdemodll.dll  ./demo/bin/demodll.o

##### 1.2 Call from Pythonl

In [2]:
from ctypes import *

flib = cdll.LoadLibrary("./demo/bin/libdemodll.dll")

# flib = windll.LoadLibrary("./demo/bin/libdemodll.dll")

## 1 argtypes,restype
flib.GetNumber.restype=c_int
flib.GetNumber.argtypes= [POINTER(c_int)]
v=c_int(10)
pi = pointer(v)
print(flib.GetNumber(pi))

## 2 Function prototypes
prototype = WINFUNCTYPE(c_int,POINTER(c_int))
f = prototype(("GetNumber", flib),)
print(f(pi))

## 3 Type conversions
f=flib.GetNumber

v=c_int(10)
print(f(byref(v)))

pi = pointer(v)
print(f(pi))

12
13
12
13


#### 2 Wrapper the Shared Library to the Python API 

SEUIF97.dll

`__stdcall` calling convention

**windll, WINFUNCTYPE**


In [ ]:
from ctypes import *

flib = windll.LoadLibrary('libseuif97.dll')
prototype = WINFUNCTYPE(c_double, c_double, c_double, c_int)

# ---(p,t) ----------------
def pt(p, t, pid):
    f = prototype(("seupt", flib),)
    result = f(p, t, pid)
    return result


def pt2h(p, t):
    f = prototype(("seupt", flib),)
    result = f(p, t, 4)
    return result

In [ ]:
h=pt(16,535,4)
h

##  2 Using C compatible data types


###  Fundamental data types

ctypes defines a number of primitive C compatible data types:

https://docs.python.org/3/library/ctypes.html#fundamental-data-types



<table class="docutils" border="1">
<colgroup>
<col width="24%">
<col width="46%">
<col width="30%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">ctypes type</th>
<th class="head">C type</th>
<th class="head">Python type</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><a title="ctypes.c_bool" class="reference internal" href="#ctypes.c_bool"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_bool</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">_Bool</span></code></td>
<td>bool (1)</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_char" class="reference internal" href="#ctypes.c_char"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_char</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">char</span></code></td>
<td>1-character bytes object</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_wchar" class="reference internal" href="#ctypes.c_wchar"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_wchar</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">wchar_t</span></code></td>
<td>1-character string</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_byte" class="reference internal" href="#ctypes.c_byte"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_byte</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">char</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ubyte" class="reference internal" href="#ctypes.c_ubyte"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ubyte</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">char</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_short" class="reference internal" href="#ctypes.c_short"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_short</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">short</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ushort" class="reference internal" href="#ctypes.c_ushort"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ushort</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">short</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_int" class="reference internal" href="#ctypes.c_int"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_int</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">int</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_uint" class="reference internal" href="#ctypes.c_uint"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_uint</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">int</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_long" class="reference internal" href="#ctypes.c_long"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_long</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ulong" class="reference internal" href="#ctypes.c_ulong"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ulong</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_longlong" class="reference internal" href="#ctypes.c_longlong"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_longlong</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">__int64</span></code> or <code class="xref c c-type docutils literal notranslate"><span class="pre">long</span> <span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ulonglong" class="reference internal" href="#ctypes.c_ulonglong"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ulonglong</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">__int64</span></code> or
<code class="xref c c-type docutils literal notranslate"><span class="pre">unsigned</span> <span class="pre">long</span> <span class="pre">long</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_size_t" class="reference internal" href="#ctypes.c_size_t"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_size_t</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">size_t</span></code></td>
<td>int</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_ssize_t" class="reference internal" href="#ctypes.c_ssize_t"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_ssize_t</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">ssize_t</span></code> or
<code class="xref c c-type docutils literal notranslate"><span class="pre">Py_ssize_t</span></code></td>
<td>int</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_float" class="reference internal" href="#ctypes.c_float"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_float</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">float</span></code></td>
<td>float</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_double" class="reference internal" href="#ctypes.c_double"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_double</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">double</span></code></td>
<td>float</td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_longdouble" class="reference internal" href="#ctypes.c_longdouble"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_longdouble</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">long</span> <span class="pre">double</span></code></td>
<td>float</td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_char_p" class="reference internal" href="#ctypes.c_char_p"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_char_p</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">char</span> <span class="pre">*</span></code> (NUL terminated)</td>
<td>bytes object or <code class="docutils literal notranslate"><span class="pre">None</span></code></td>
</tr>
<tr class="row-odd"><td><a title="ctypes.c_wchar_p" class="reference internal" href="#ctypes.c_wchar_p"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_wchar_p</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">wchar_t</span> <span class="pre">*</span></code> (NUL terminated)</td>
<td>string or <code class="docutils literal notranslate"><span class="pre">None</span></code></td>
</tr>
<tr class="row-even"><td><a title="ctypes.c_void_p" class="reference internal" href="#ctypes.c_void_p"><code class="xref py py-class docutils literal notranslate"><span class="pre">c_void_p</span></code></a></td>
<td><code class="xref c c-type docutils literal notranslate"><span class="pre">void</span> <span class="pre">*</span></code></td>
<td>int or <code class="docutils literal notranslate"><span class="pre">None</span></code></td>
</tr>
</tbody>
</table>
 
All these types can be created by calling them with an optional initializer of the correct type and value:

In [11]:
i = c_int(42)
print(i)
i.value

c_long(42)


42

### The Example

The examples show that how to use **C compatible data type** to call functions in DLLs

* Structure

* Arrays(one/two-dimensional)

####  The Functions in C

In [ ]:
%%file ./demo/src/democtypes.c

// 1 Struct
typedef struct SimpleStruct1
{
    int nNo;
    float fValue;
} SimpleStruct;

__declspec(dllexport)  int __stdcall  TestSimpleStruct(int *n,int *m,SimpleStruct struin,SimpleStruct *struout)
{
  struout->fValue= struin.fValue+2;
  struout->nNo=struin.nNo+3;
  
  *n=struin.nNo+20;
  *m=*n+30;
  return struout->nNo;
}

// 2 using the name of one-dimensional array in Python，non byref
__declspec(dllexport)  void  __stdcall  TestArray1(int nsize,double *narray)
{
  for(int i=0; i<nsize; i++)
  {
     narray[i]=i*2.3;  
  }
}

// 3 using the name of two-dimensional array in Python，  non byref
__declspec(dllexport)  void  __stdcall  TestArray21(int ni, int nj,double *ptr)
{
    int i, j;
    for(i=0; i<ni; i++)
    {
        for(j=0; j<nj; j++)
        {
            ptr[i*ni+j]= ptr[i*ni+j]+i*ni + j;
        }
	}       
}

// 4 using byref in Python
__declspec(dllexport)  void  __stdcall  TestArray22(int ni, int nj,double **ptr)
{
    int i, j;
    for(i=0; i<ni; i++)
    {
        for(j=0; j<nj; j++)
        {
            ptr[i][j]=ptr[i][j]+i*ni + j;
        }
	} 
}

In [ ]:
%%file ./demo/makefile-democtypesdll

all: libdemoctypes

libdemoctypes: obj
	 gcc  -shared -o ./demo/bin/libdemoctypes.dll democtypes.o
	 del democtypes.o

obj: 
	 gcc -c -O3 -Wall -fPIC  ./demo/src/democtypes.c 

In [ ]:
!make -f ./demo/makefile-democtypesdll

####  Using C compatible data types to Call DLL

##### 1 Loading dynamic link libraries 

`windll` loads libraries which export functions using the `stdcall` calling convention 

In [ ]:
from ctypes import *

flib=windll.LoadLibrary("./demo/bin/libdemoctypes.dll")

#### 2  using ctypes to call DLL : Structures

[Structures](https://docs.python.org/3/library/ctypes.html#structures-and-unions) must derive from the `Structure` base classes which are defined in the `ctypes` module.

Each subclass must define a `_fields_` attribute.

`_fields_` must be `a list of 2-tuples`, containing a field `name` and a field `type`: 

```python
_fields_=[(a field name,a field `type`)]
```

In [ ]:
class SimpStruct(Structure):
    _fields_ = [("nNo", c_int),
                ("fValue", c_float)]

#  TestSimpleStruct
f1=flib.TestSimpleStruct
simplein = SimpStruct()
simplein.nNo = 16
simplein.fValue = 3.14

simpleout = SimpStruct()

n=c_int()
m=c_int()
nNo=f1(byref(n),byref(m),simplein,byref(simpleout))

print(simpleout.fValue)
print(nNo)
print(n.value)
print(m.value)

##### 3  using ctypes to call DLL :one/two-dimensional Array

[Arrays](https://docs.python.org/3/library/ctypes.html#arrays) are sequences, containing a fixed number of instances of the same type.

The recommended way to create array types is by multiplying a data type with a positive integer:

In [ ]:
narray=(c_double*10)()  
type(narray)

In [ ]:
list1=[1,2,3]
narray=(c_double*len(list1))(*list1)
print(narray,narray[1])

In [ ]:
#  one-dimensional array
f2=flib.TestArray1
na=c_int()
na=10

narray=(c_double*10)()  

# using the name of one-dimensional array,non byref
f2(na,narray)
print(narray[8])

#  two-dimensional array  
f3=flib.TestArray21
ptr = (c_double*5*4)()
# two-dimensional array: j*i
for i in range(4):
    for j in range(5):
        ptr[i][j] = i + j
        
# using the name of two-dimensional array,non byref，
f3(4,5,ptr)
print(ptr[1][3])

# two-dimensional array
f4=flib.TestArray22
indata = (POINTER(c_double) * 5)()
for i in range(5):
    # Allocate arrays of double
    indata[i] = (c_double * 6)()
    for j in range(6):
        indata[i][j] = 1.0

f4(5,6,byref(indata))
print(indata[1][3])


## Reference

Python ctypes http://docs.python.org/3/library/ctypes.html

C-Types Foreign Function Interface (numpy.ctypeslib)
  https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html
  